# Tutorial 1: Getting Started with SQuADDS

In this tutorial, we will walk you through some basic usage of SQuADDS. By the end of this tutorial, you will be able to:

- Have an HuggingFace account

- Access the SQuADDS Database

- Use the SQuADDS API to query for closest and "best-guess" interpolated device designs for your chosen Hamiltonian parameters

- Simulate the "best-guess" design using an EM solver tool

In [ ]:
! conda list | grep squadds

In [1]:
%load_ext autoreload
%autoreload 2

Since the SQuADDS Database is hosted on [HuggingFace](https://huggingface.co/), we will need to create an account and get an API key to access the database.

## HuggingFace 🤗 

HuggingFace is a company that provides a large number of NLP models and datasets. They also provide a platform to host your own models and datasets.

### Creating an  Account

Follow the instructions here - [HuggingFace: Sign Up](https://huggingface.co/join) - to create an account.

Once you have created an account, you can get your API key from the [settings page](https://huggingface.co/settings/token).

Please update the `HUGGINGFACE_API_KEY` variable in the `.env` file with your API key **OR** execute the following code to set the environment variable.

In [ ]:
from squadds.core.utils import set_huggingface_api_key

set_huggingface_api_key()

### Login

To login to your HuggingFace account, run the following command in your terminal:

```bash
huggingface-cli login
```



You will be prompted to enter your username and password. Once you have logged in, you can check your login status by running the following command:

```bash
huggingface-cli whoami
```

### Accessing the SQuADDS Database using the HuggingFace API

The SQuADDS Database is hosted on HuggingFace. You can access the database using the `datasets` library from HuggingFace.

In [ ]:
from datasets import get_dataset_config_names
from datasets import load_dataset

configs = get_dataset_config_names("SQuADDS/SQuADDS_DB")

You can navigate the database using HuggingFace API. For example, you can access the qubit database using the following code:

In [ ]:
qubit_data = load_dataset("SQuADDS/SQuADDS_DB", configs[0])

In [ ]:
qubit_data

Each configuration in the dataset is uniquely identified by their `config`. For the SQuADDS Database, the `config` string is created in the following format:

```python
config = f"{component}_{component_name}_{data_type}"
```

where `component` is the name of the component, `component_name` is the name of the component (in Qiskit Metal), and `data_type` is the type of simulation data that has been contributed.

Lets check what the `config` string looks like for our database:

In [ ]:
components = []
component_names = []
data_types = []

for config in configs:
    components.append(config.split("-")[0])
    component_names.append(config.split("-")[1])
    data_types.append(config.split("-")[2])
    
print(components)
print(component_names)
print(data_types)
    

## Using the SQuADDS API to access and anlyze the database

While it is possible to directly access the SQuADDS Database using the `datasets` library, we have created a simple API to make it easier to query the database.

The main object we use to access the database is the `SQuADDS_DB` class. 

In [2]:
from squadds import SQuADDS_DB

db = SQuADDS_DB()

DotNet Core correctly loaded.


You can get a summary of the datasets by running.

In [ ]:
db.view_datasets()

To check for the `config` names

In [ ]:
db.get_configs()

If you are interested to learn more about each configuration, you can do so by using the `get_dataset_info` method.


In [ ]:
db.get_dataset_info(component="qubit", component_name="TransmonCross", data_type="cap_matrix")

In [ ]:
db.get_dataset_info(component="cavity_claw", component_name="RouteMeander", data_type="eigenmode")

You can get the entire dataset as a Pandas DataFrame by using the `get_dataset` method.

In [ ]:
db.see_dataset(component="qubit", component_name="TransmonCross", data_type="cap_matrix")

If you want to learn about the who contributed the data, you can use the following methods:

In [ ]:
db.view_contributors_of("qubit", "TransmonCross", "cap_matrix")

In [ ]:
db.view_all_contributors()

As the SQuADDS_DB dataset updates, so will all the information we have queried automatically. 

## Making Systems out of Circuit QED Elements

One of the main use cases of the SQuADDS project is to get the design space parameters for systems of our interest using our desired Hamiltonian parameters.

Using the SQuADDS API, we can "build" a system by choosing the circuit QED components we want to use.

The following subsections walks you through some examples.

### Querying for the a target qubit design

Let's say you know the Hamiltonian parameters of a qubit you want to use. You can use the SQuADDS API to query for the closest design to your target qubit.

We first need to select our sytem of interest.

In [5]:
db.select_system("qubit")

Now, we need to specify to SQuADDS what type of `qubit` our system is. We can do this by using the `select_qubit` method.

In [6]:
db.select_qubit("TransmonCross")

We now create the system dataframe so that we can query for the design parameters we are interested in.

In [7]:
df = db.create_system_df()
df

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

,design_options,design_tool,claw_to_claw,claw_to_ground,cross_to_claw,cross_to_cross,cross_to_ground,ground_to_ground,units,PI,date_created,group,institution,uploader,renderer_options,setup,simulator
0,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,fF,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS
1,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,82.44280,79.19378,2.93820,188.15089,188.15089,333.52997,fF,"Eli Levenson-Falk, PhD",2023-10-25-153123,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS
2,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,83.76412,80.18130,3.16131,104.35340,104.35340,237.02548,fF,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS
3,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,103.37057,97.22405,5.77590,174.13928,174.13928,335.31609,fF,"Eli Levenson-Falk, PhD",2023-10-25-153126,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS
4,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,68.92854,65.68607,2.87375,120.03923,120.03923,240.34085,fF,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,106.43025,101.53197,4.45645,174.46380,174.46380,340.62919,fF,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS
1930,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,121.10943,112.62570,7.95178,187.43537,187.43537,367.34003,fF,"Eli Levenson-Falk, PhD",2023-09-20-142549,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS
1931,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,144.56289,136.36810,7.65968,172.14561,172.14561,372.39970,fF,"Eli Levenson-Falk, PhD",2023-10-25-153123,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS
1932,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,68.76413,65.78116,2.48795,56.75230,56.75230,166.57383,fF,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS


Now that we have created our system dataframe, we can query for the closest design to our target qubit parameters. To do this we need to call the `Analyzer` object.

In [8]:
from squadds import Analyzer

We instatantaite the `Analyzer` object by passing in the `SQuADDS_DB` instance we created earlier.

In [9]:
analyzer = Analyzer(db)

We can now check for what type of Hamiltonian parameters are available for our chosen system

In [ ]:
analyzer.target_param_keys()

Now, Let's select a geometry which results in the closest qubit characteristics

Call `Analyzer.find_closest`

**Documentation:**

```
Finds the rows in the DataFrame with the closest matching characteristics
to the given target parameters using a specified metric.

Args:
    target_params (dict): A dictionary containing the target values for columns in `self.df`.
                          Keys are column names and values are the target values.
    num_top (int): The number of closest matching rows to return.
    metric (str, optional): The distance metric to use for finding the closest matches.
                            Available options are specified in `self.__supported_metrics__`.
                            Defaults to 'Euclidean'.
    display (bool, optional): Whether to display warnings and logs. Defaults to True.

Returns:
    pd.DataFrame: A DataFrame containing the rows with the closest matching characteristics,
                  sorted by the distance metric.

Raises:
    ValueError: If the specified metric is not supported or `num_top` exceeds the DataFrame size.
```

You are given the choice of the following metrics.

In [ ]:
analyzer.__supported_metrics__

Define your Hamiltonian parameters that you want to use for your qubit 

In [ ]:
target_params={"qubit_frequency_GHz": 4, "anharmonicity_MHz": -200}

In [ ]:
results = analyzer.find_closest(target_params=target_params,
                                       num_top=3,
                                       metric="Euclidean",
                                       display=True)
results

Thats it! You have now found some designs for your qubit that are closest to your target Hamiltonian parameters.

#### Using Custom Metrics

To use a custom metric first define the function. For example, lets say we want the manhattan metric

In [ ]:
def manhattan_distance(target, simulated):
    return sum(abs(target[key] - simulated.get(key, 0)) for key in target)

In [ ]:
analyzer.custom_metric_func = manhattan_distance

In [ ]:
analyzer.find_closest(target_params=target_params,
                                            num_top=1,
                                            metric="Custom",
                                            display=True)

In [ ]:
best_options = results.iloc[0]["design_options"]
best_options

You can pass in the `design_options` from the closest design to the `options` argument of your selected qubit and render it in qiskit metal.

In [ ]:
# Qiskit Metal imports
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict
from qiskit_metal.designs.design_multiplanar import MultiPlanar

from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.core import QRoute, QRoutePoint

In [ ]:
design = MultiPlanar(metadata={},
                     overwrite_enabled=True)
gui = MetalGUI(design)

In [ ]:
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross

TransmonCross(design, "transmon", options=best_options)

gui.rebuild()
gui.zoom_on_components(['transmon'])
gui.screenshot("qubit_only.png")

### Querying for a target cavity design

The same workflow can be used to query for a target cavity design.

While it is not necessary, it may be a good idea to `unselect_all()` before creating a new system.

In [ ]:
db.unselect_all()

Proceed with selecting the system of interest

In [ ]:
db.select_system("cavity_claw")

In [ ]:
db.select_cavity_claw("RouteMeander")

In [ ]:
db.select_coupler("CLT")

It's always a good idea to check that the system you have selected is correct.

In [ ]:
db.show_selections()

Great! lets create the system dataframe and analyze it.

In [ ]:
df = db.create_system_df()

In [ ]:
analyzer = Analyzer(db)

In [ ]:
analyzer.target_param_keys()

Select the Hamiltonian parameters you want to use for your cavity and search for the closest designs.

In [ ]:
target_params = {"cavity_frequency_GHz": 6.9,
                "kappa_kHz": 120,
                "resonator_type":"quarter"}

In [ ]:
results = analyzer.find_closest(target_params=target_params,
                                       num_top=5,
                                       metric="Euclidean",
                                       display=True)
results

Similarly, if we wanted a `half`-wave resonator (i.e. has an `NCap` coupling to the feedline), we can do the following:

In [ ]:
db.unselect_all()

db.select_system("cavity_claw")
db.select_cavity_claw("RouteMeander")
db.select_coupler("NCap")

In [ ]:
df = db.create_system_df()

In [ ]:
analyzer = Analyzer(db)

In [ ]:
target_params = {"cavity_frequency_GHz": 6.9,
                "kappa_kHz": 120,
                "resonator_type":"half"}

In [12]:
cavity_df = db.get_dataset(data_type="eigenmode", component="cavity_claw", component_name="RouteMeander")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
cavity_df

,coupler_type,design_options,design_tool,resonator_type,cavity_frequency,kappa,units,PI,date_created,group,institution,misc,uploader,renderer_options,setup,simulator
0,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,5.353550e+09,161106.598429,Hz,"Eli Levenson-Falk, PhD",2023-12-01-170608,LFL,USC,None,Andre Kuo,None,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS
1,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,8.399241e+09,268412.116632,Hz,"Eli Levenson-Falk, PhD",2023-12-04-124953,LFL,USC,None,Andre Kuo,None,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS
2,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,8.694845e+09,255873.654612,Hz,"Eli Levenson-Falk, PhD",2023-12-09-204334,LFL,USC,None,Andre Kuo,None,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS
3,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,6.616574e+09,30459.761161,Hz,"Eli Levenson-Falk, PhD",2023-12-08-173545,LFL,USC,None,Andre Kuo,None,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS
4,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,7.986835e+09,208304.221064,Hz,"Eli Levenson-Falk, PhD",2023-12-09-204334,LFL,USC,None,Andre Kuo,None,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,NCap,{'claw_opts': {'connection_pads': {'readout': ...,Qiskit Metal,half,1.859429e+10,1708.750004,Hz,Eli Levenson-Falk,2024-02-26 230413,LFL,USC,,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS
906,NCap,{'claw_opts': {'connection_pads': {'readout': ...,Qiskit Metal,half,1.609044e+10,1478.741367,Hz,Eli Levenson-Falk,2024-02-26 230413,LFL,USC,,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS
907,NCap,{'claw_opts': {'connection_pads': {'readout': ...,Qiskit Metal,half,7.502149e+09,689.346880,Hz,Eli Levenson-Falk,2024-02-26 230413,LFL,USC,,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS
908,NCap,{'claw_opts': {'connection_pads': {'readout': ...,Qiskit Metal,half,7.064153e+09,649.104744,Hz,Eli Levenson-Falk,2024-02-26 230413,LFL,USC,,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS


In [13]:
ncap_df = db.get_dataset(data_type="cap_matrix", component="coupler", component_name="NCap") #TODO: handle dynamically

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
ncap_df

,coupler_type,design_options,design_tool,bottom_to_bottom,bottom_to_ground,ground_to_ground,top_to_bottom,top_to_ground,top_to_top,units,PI,date_created,group,institution,uploader,setup,simulator
0,NCap,"{'cap_distance': '50.9um', 'cap_gap': '2.1um',...",qiskit-metal,12.78614,12.09846,58.45017,0.51663,14.95437,15.65491,nH,"Eli Levenson-Falk, PhD",2023-11-09-011049,LFL,USC,Andre Kuo,"{'auto_increase_solution_order': 'False', 'ena...",Ansys HFSS
1,NCap,"{'cap_distance': '50.9um', 'cap_gap': '2.1um',...",qiskit-metal,94.96956,58.07364,124.18885,36.28408,29.22434,65.85844,nH,"Eli Levenson-Falk, PhD",2023-11-09-011049,LFL,USC,Andre Kuo,"{'auto_increase_solution_order': 'False', 'ena...",Ansys HFSS
2,NCap,"{'cap_distance': '50.9um', 'cap_gap': '4.1um',...",qiskit-metal,21.94864,18.77498,66.60369,2.88026,15.65588,18.77838,nH,"Eli Levenson-Falk, PhD",2023-11-09-011049,LFL,USC,Andre Kuo,"{'auto_increase_solution_order': 'False', 'ena...",Ansys HFSS
3,NCap,"{'cap_distance': '50.9um', 'cap_gap': '2.1um',...",qiskit-metal,55.18578,33.69106,88.75992,21.05076,20.98596,42.46632,nH,"Eli Levenson-Falk, PhD",2023-11-09-011049,LFL,USC,Andre Kuo,"{'auto_increase_solution_order': 'False', 'ena...",Ansys HFSS
4,NCap,"{'cap_distance': '50.9um', 'cap_gap': '4.1um',...",qiskit-metal,60.57606,41.62987,101.33585,18.40010,24.77954,43.49158,nH,"Eli Levenson-Falk, PhD",2023-11-09-011049,LFL,USC,Andre Kuo,"{'auto_increase_solution_order': 'False', 'ena...",Ansys HFSS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,NCap,"{'cap_distance': '50.9um', 'cap_gap': '2.1um',...",qiskit-metal,15.86284,14.59008,64.32406,1.02606,18.87248,20.16504,nH,"Eli Levenson-Falk, PhD",2023-11-09-011049,LFL,USC,Andre Kuo,"{'auto_increase_solution_order': 'False', 'ena...",Ansys HFSS
426,NCap,"{'cap_distance': '50.9um', 'cap_gap': '4.1um',...",qiskit-metal,28.43531,24.50768,74.50121,3.57699,17.41757,21.23155,nH,"Eli Levenson-Falk, PhD",2023-11-09-011049,LFL,USC,Andre Kuo,"{'auto_increase_solution_order': 'False', 'ena...",Ansys HFSS
427,NCap,"{'cap_distance': '50.9um', 'cap_gap': '4.1um',...",qiskit-metal,55.36257,38.54199,102.24635,16.38109,28.66939,45.47336,nH,"Eli Levenson-Falk, PhD",2023-11-09-011049,LFL,USC,Andre Kuo,"{'auto_increase_solution_order': 'False', 'ena...",Ansys HFSS
428,NCap,"{'cap_distance': '50.9um', 'cap_gap': '4.1um',...",qiskit-metal,23.20076,19.39282,65.27181,3.52960,14.36617,18.15142,nH,"Eli Levenson-Falk, PhD",2023-11-09-011049,LFL,USC,Andre Kuo,"{'auto_increase_solution_order': 'False', 'ena...",Ansys HFSS


In [17]:
ncap_opts = ncap_df.iloc[0]["design_options"]
ncap_opts

{'cap_distance': '50.9um',
 'cap_gap': '2.1um',
 'cap_gap_ground': '5.1um',
 'cap_width': '4.9um',
 'finger_count': '1',
 'finger_length': '25.9um',
 'orientation': '-90',
 'prime_gap': '5.1um',
 'prime_width': '11.7um',
 'second_gap': '5.1um',
 'second_width': '11.7um'}

In [20]:
cpw_cplr_opts = cavity_df.iloc[-1]["design_options"]["cplr_opts"]
cpw_cplr_opts

{'cap_distance': '50.9um',
 'cap_gap': '5um',
 'cap_gap_ground': '5.1um',
 'cap_width': '10um',
 'coupling_length': None,
 'coupling_space': None,
 'down_length': None,
 'finger_count': '5',
 'finger_length': '30um',
 'open_termination': None,
 'orientation': '-90',
 'prime_gap': '5.1um',
 'prime_width': '11.7um',
 'second_gap': '5.1um',
 'second_width': '11.7um'}

In [21]:
common_terms = set(ncap_opts.keys()) & set(cpw_cplr_opts.keys())
common_terms


{'cap_distance',
 'cap_gap',
 'cap_gap_ground',
 'cap_width',
 'finger_count',
 'finger_length',
 'orientation',
 'prime_gap',
 'prime_width',
 'second_gap',
 'second_width'}

In [22]:
# Function to merge dataframes based on specific common design options
def merge_dataframes_specific_common_terms(cavity_df, ncap_df, common_terms):
    # Create temporary columns in both dataframes for the specific common terms
    for term in common_terms:
        cavity_df[f'temp_{term}'] = cavity_df['design_options'].apply(lambda x: x['cplr_opts'].get(term))
        ncap_df[f'temp_{term}'] = ncap_df['design_options'].apply(lambda x: x.get(term))
    
    # Perform the merge on the specific common terms
    merged_df = pd.merge(cavity_df, ncap_df, 
                         left_on=[f'temp_{term}' for term in common_terms], 
                         right_on=[f'temp_{term}' for term in common_terms],
                         suffixes=('', '_ncap'))

    # Now, select only the subset of cavity_df and add additional columns from ncap_df
    additional_columns = ['bottom_to_bottom', 'bottom_to_ground', 'ground_to_ground',
                          'top_to_bottom', 'top_to_ground', 'top_to_top', 'units_ncap']
    merged_df = merged_df[[col for col in cavity_df.columns if col not in ['design_options']] + additional_columns]

    # Drop the temporary columns used for merging
    merged_df.drop(columns=[f'temp_{term}' for term in common_terms], inplace=True)

    return merged_df

# Define the specific common terms we're interested in for the merge
#specific_common_terms = ['prime_width', 'prime_gap', 'second_width', 'second_gap']
specific_common_terms = ['prime_width', 'prime_gap']


In [23]:
# Merge the dataframes based on the specific common terms
# filter ncap_df to only include 5 random rows from it
ncap_df_sample = ncap_df.sample(1)


In [24]:
merged_df = merge_dataframes_specific_common_terms(ncap_cavity_df, ncap_df_sample, specific_common_terms)
merged_df

NameError: name 'ncap_cavity_df' is not defined

In [ ]:
%matplotlib inline

In [ ]:
# Extracting values for initial common terms from cavity_df
cavity_values = {}
for term in initial_common_terms:
    cavity_values[term] = cavity_df['design_options'].apply(lambda x: x['cplr_opts'][term])

In [ ]:

import matplotlib.pyplot as plt

# Extracting initial common terms for statistics
initial_common_terms = list(common_terms)

# Creating plots
fig, axs = plt.subplots(len(initial_common_terms), 2, figsize=(12, 10), sharey='row')
fig.suptitle('Statistics of Initial Common Terms between ncap_df and cavity_df')

for i, term in enumerate(initial_common_terms):
    # Plotting for ncap_df
    axs[i, 0].hist(ncap_values[term], color='skyblue', edgecolor='black', alpha=0.7)
    axs[i, 0].set_title(f'ncap_df: {term}')
    axs[i, 0].set_ylabel('Frequency')

    try:
        # Plotting for cavity_df
        axs[i, 1].hist(cavity_values[term], color='orange', edgecolor='black', alpha=0.7)
        axs[i, 1].set_title(f'cavity_df: {term}')
    except:
        pass

# Adjusting layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
results = analyzer.find_closest(target_params=target_params,
                                       num_top=5,
                                       metric="Euclidean",
                                       display=True)
results

Lets say we want to use the "Weighted Euclidean" metric to find the closest design to our target cavity parameters.

#### Weighted Euclidean Metric

You can do a weighted Euclidean metric instead.

$$
F(\{P_i\},\{p_i\}) = \sum_i w_i\frac{(P_i - p_i)^2}{P_i^2}
$$

Here \( w_i \) are weights which default to 1 if not user-defined.

Note: The default metric for `find_closest` is `Euclidean` when not user-defined.


In [ ]:
# Set up the weights
analyzer.metric_weights = {"cavity_frequency_GHz": 2, "kappa_kHz": 1}

In [ ]:
results = analyzer.find_closest(target_params=target_params,
                                       num_top=3,
                                       metric="Weighted Euclidean",
                                       display=True)
results

### Querying for a target qubit-cavity design

Again, we follow the same procedure as before.

In [ ]:
db.select_system(["qubit","cavity_claw"])

In [ ]:
db.select_qubit("TransmonCross")
db.select_cavity_claw("RouteMeander")
db.select_coupler("CLT")

In [ ]:
db.show_selections()

In [ ]:
merged_df = db.create_system_df()

In [ ]:
merged_df

Pass the `SQuADDS_DB` instance to the `Analyzer` object.

In [ ]:
analyzer = Analyzer(db)

Always good to check whether the system you have selected is correct.

In [ ]:
db.selected_system

In [ ]:
analyzer.selected_system

Define the `target_params` for your qubit-cavity system.

In [ ]:
target_params = {
                "qubit_frequency_GHz": 4,
                "cavity_frequency_GHz": 6.2,
                "kappa_kHz": 120,
                "resonator_type":"quarter",
                "anharmonicity_MHz": -200,
                "g_MHz": 70}

In [ ]:
results = analyzer.find_closest(target_params=target_params,
                                       num_top=3,
                                       metric="Euclidean",
                                       display=True)
results

Awesome! we have some designs for our qubit-cavity system. To see where the closest design lies in the Hamiltonian parameter space, we can use the `closest_design_in_H_space` method.

In [ ]:
%matplotlib inline

In [ ]:
analyzer.closest_design_in_H_space()

#### Interpolation of Best Designs

Even though the `closest_design` will become better as more validated pre-simulated points are added to the database, it is still a good idea to interpolate to get the best designs.

We use the physics inspired interpolation algorithm described in our [paper](https://arxiv.org/pdf/2312.13483.pdf) - `ScalingInterpolator` class to interpolate the best designs.

In [ ]:
from squadds.interpolations.physics import ScalingInterpolator

We pass the `Analzyer` object and the `target_params` dict to the `ScalingInterpolator` class.

In [ ]:
# Create an instance of ScalingInterpolator
interpolator = ScalingInterpolator(analyzer, target_params)

design_df = interpolator.get_design()

The `design_df` contains the various `design_options` for the best designs and also the `sim_options` needed to simulate them.

In [ ]:
design_df

Congrats for making it to the end of this tutorial! 🤗🎉 You have now learned how to use the SQuADDS API to query for closest and "best-guess" interpolated device designs for your chosen Hamiltonian parameters.

## Next Steps...

In the next [tutorial](https://lfl-lab.github.io/SQuADDS/source/tutorials/Tutorial-2_Simulate_interpolated_designs.html), we will learn how to simulate the "best-guess" design using an EM solver tool and the `SQuADDS` API.

## License

<div style='width: 100%; background-color:#3cb1c2;color:#324344;padding-left: 10px; padding-bottom: 10px; padding-right: 10px; padding-top: 5px'>
    <h3>This code is a part of SQuADDS</h3>
    <p>Developed by Sadman Ahmed Shanto</p>
    <p>&copy; Copyright Sadman Ahmed Shanto & Eli Levenson-Falk 2023.</p>
    <p>This code is licensed under the MIT License. You may<br> obtain a copy of this license in the LICENSE.txt file in the root directory<br> of this source tree.</p>
    <p>Any modifications or derivative works of this code must retain this<br>copyright notice, and modified files need to carry a notice indicating<br>that they have been altered from the originals.</p>
</div>